# Flame AI


## Libraries

In [21]:
import cv2
import numpy as np
import pandas as pd
import colorsys
import sys
import os
import time
from matplotlib import pyplot as plt

from tqdm import tqdm
from PIL import Image

import logging
from logging.handlers import RotatingFileHandler




## Contour Functions


In [17]:
import os
import cv2
import numpy as np
import pandas as pd
import time
import logging

logger = logging.getLogger(__name__)

def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def merge_contours(contours, max_distance=70):
    merged_contours = []
    used = [False] * len(contours)

    def get_centroid(contour):
        M = cv2.moments(contour)
        if M['m00'] == 0:
            return None
        return (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))

    for i, c1 in enumerate(contours):
        if used[i]:
            continue
        merged = [c1]
        centroid1 = get_centroid(c1)
        if centroid1 is None:
            continue
        for j, c2 in enumerate(contours):
            if i != j and not used[j]:
                centroid2 = get_centroid(c2)
                if centroid2 is None:
                    continue
                dist = np.linalg.norm(np.array(centroid1) - np.array(centroid2))
                if dist < max_distance:
                    merged.append(c2)
                    used[j] = True
        merged = np.vstack(merged)
        merged_contours.append(merged)
        used[i] = True

    return merged_contours

def extract_features(frame):
    img = frame[:, :, 1]  # Use the green channel for analysis
    # Apply masking to the edges of the frame to avoid edge effects
    img[0:40, :] = 0
    img[frame.shape[0] - 21:frame.shape[0] - 1, :] = 0
    img[:, 0:40] = 0
    img[:, frame.shape[1] - 21:frame.shape[1] - 1] = 0

    # Apply binary thresholding and morphological operations to clean up the image
    _, B2 = cv2.threshold(img, 60, 255, cv2.THRESH_BINARY)
    B2 = cv2.morphologyEx(B2, cv2.MORPH_CLOSE, np.ones((10, 10)))

    # Find and merge contours
    contours, _ = cv2.findContours(B2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = merge_contours(contours)

    features = {}
    for contour_id, c in enumerate(contours, start=1):
        if cv2.contourArea(c) > 100:
            rect = cv2.minAreaRect(c)
            center, size, angle = rect
            width, height = size
            area = width * height
            contour_area = cv2.contourArea(c)

            # Calculate the area ratio
            area_ratio = contour_area / area if area != 0 else 0

            features[f'centerX_{contour_id}'] = center[0]
            features[f'centerY_{contour_id}'] = center[1]
            features[f'width_{contour_id}'] = width
            features[f'height_{contour_id}'] = height
            features[f'angle_{contour_id}'] = angle
            features[f'rect_area_{contour_id}'] = area
            features[f'contour_area_{contour_id}'] = contour_area
            features[f'area_ratio_{contour_id}'] = area_ratio  # Add the new feature

    return features, B2, contours

def match_contours(previous_contours, current_contours, max_distance=50):
    matches = {}
    used = [False] * len(previous_contours)

    def get_centroid(contour):
        M = cv2.moments(contour)
        if M['m00'] == 0:
            return None
        return (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))

    for i, current_contour in enumerate(current_contours):
        current_centroid = get_centroid(current_contour)
        if current_centroid is None:
            continue

        best_match_index = None
        best_match_distance = max_distance

        for j, previous_contour in enumerate(previous_contours):
            if used[j]:
                continue

            previous_centroid = get_centroid(previous_contour)
            if previous_centroid is None:
                continue

            distance = np.linalg.norm(np.array(current_centroid) - np.array(previous_centroid))

            if distance < best_match_distance:
                best_match_distance = distance
                best_match_index = j

        if best_match_index is not None:
            matches[i] = best_match_index
            used[best_match_index] = True

    return matches

def process_video(video_path):
    logger.info(f"Processing video: {video_path}")
    cap = cv2.VideoCapture(video_path)
    all_features = []
    frame_count = 0

    # Create a window for display.
    cv2.namedWindow("Video Preview", cv2.WINDOW_NORMAL)

    previous_contours = []
    object_id_map = {}
    next_object_id = 1

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        frame_features, B2, current_contours = extract_features(frame)
        
        matches = match_contours(previous_contours, current_contours)
        new_object_id_map = {}
        
        for current_index, contour in enumerate(current_contours):
            if current_index in matches:
                previous_index = matches[current_index]
                object_id = object_id_map[previous_index]
            else:
                object_id = next_object_id
                next_object_id += 1
            
            new_object_id_map[current_index] = object_id

            rect = cv2.minAreaRect(contour)
            center, size, angle = rect
            width, height = size
            area = width * height
            contour_area = cv2.contourArea(contour)
            area_ratio = contour_area / area if area != 0 else 0

            frame_features[f'centerX_{object_id}'] = center[0]
            frame_features[f'centerY_{object_id}'] = center[1]
            frame_features[f'width_{object_id}'] = width
            frame_features[f'height_{object_id}'] = height
            frame_features[f'angle_{object_id}'] = angle
            frame_features[f'rect_area_{object_id}'] = area
            frame_features[f'contour_area_{object_id}'] = contour_area
            frame_features[f'area_ratio_{object_id}'] = area_ratio

        previous_contours = current_contours
        object_id_map = new_object_id_map
        all_features.append(frame_features)

        # Draw each contour on the frame
        for contour in current_contours:
            color = (0, 255, 0)  # Green color for contours
            cv2.drawContours(frame, [contour], -1, color, 2)

        # Display the frame with contours
        cv2.imshow("Video Preview", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):  # Allow exit with 'q'
            break

    cap.release()
    cv2.destroyAllWindows()
    df = pd.DataFrame(all_features)
    return df

def process_all_videos(video_folder):
    for file in os.listdir(video_folder):
        if file.endswith('.mp4'):
            video_path = os.path.join(video_folder, file)
            start_time = time.time()
            features = process_video(video_path)
            end_time = time.time()
            processing_time = end_time - start_time

            logger.info(f"Processed {file} in {processing_time:.2f} seconds")

            if not features.empty:
                csv_file_path = os.path.join(video_folder, f"{os.path.splitext(file)[0]}_features.csv")
                features.to_csv(csv_file_path, index=False)
                logger.info(f"Features saved to {csv_file_path}")


## Vector Functions 

In [13]:


def process_csv(file_path, output_dir):
    # Load the CSV file
    df = pd.read_csv(file_path)
    
    # Drop rows where all elements are NaN
    df = df.dropna(how='all')
    
    # Define the window size and overlap
    window_size = 20
    overlap = 19

    # Create a directory for the current CSV's output
    base_filename = os.path.splitext(os.path.basename(file_path))[0]
    current_output_dir = os.path.join(output_dir, base_filename)
    if not os.path.exists(current_output_dir):
        os.makedirs(current_output_dir)
    
    # Process each column
    for column in df.columns:
        # Drop NaN values from the column
        clean_data = df[column].dropna().reset_index(drop=True)
        
        # Slide window through the data
        num_windows = len(clean_data) - window_size + 1
        for start in range(num_windows):
            end = start + window_size
            window_data = clean_data[start:end]
            
            # Normalize the data in the window except for 'area_ratio'
            if column != 'area_ratio':
                max_value = window_data.max()
                normalized_data = window_data / max_value if max_value > 0 else window_data
                vector_file_name = f"{column}_vector_{start}.csv"
            else:
                # For 'area_ratio', just pass the data as is without normalization
                normalized_data = window_data
                vector_file_name = f"ratio_vector_{start}.csv"  # Custom file name for 'area_ratio'
            
            # Save the window data to a CSV file
            vector_file_path = os.path.join(current_output_dir, vector_file_name)
            normalized_data.to_csv(vector_file_path, index=False, header=False)

def process_all_csvs(input_dir, output_dir):
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Process each CSV file in the directory
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.csv'):
            file_path = os.path.join(input_dir, file_name)
            process_csv(file_path, output_dir)


## Labeled Data CSV Functions


In [14]:

def load_data_from_folder(folder_path):
    data_frames = []
    parameters = {
        'area_ratio': 1,
        'contour_area': 2,
        'rect_area': 3,
        'angle': 8,
        'height': 4,
        'width': 5,
        'centerX': 6,
        'centerY': 7
    }
    
    for parameter, label in parameters.items():
        print(f"Loading files for parameter: {parameter}")
        
        # Initialize tqdm progress bar
        parameter_files = [filename for filename in os.listdir(folder_path) if parameter in filename]
        for filename in tqdm(parameter_files, desc=f"Processing {parameter} files"):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path, header=None)
            
            # Ensure we have 20 data points per file
            if len(df) != 20:
                raise ValueError(f"File {filename} does not have 20 data points.")
            
            df = df.T  # Transpose the DataFrame
            df.columns = [f'value_{i}' for i in range(1, 21)]
            df['label'] = label
            data_frames.append(df)

    combined_df = pd.concat(data_frames, ignore_index=True)
    return combined_df

def prepare_data(folder_path):
    # Load data
    df = load_data_from_folder(folder_path)
    
    return df




## Main code


### Creating Full CSV From Videos

In [20]:


log_file_path = r"C:\Users\thaim\OneDrive\Desktop\test for flame ai\process_08_07_experiment.log"
logger = logging.getLogger("FlameDetector")
logger.setLevel(logging.INFO)
handler = RotatingFileHandler(log_file_path, maxBytes=5*1024*1024, backupCount=2)  # 5 MB per file, max 2 files
formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)


video_folder = r"C:\Users\thaim\OneDrive\Desktop\test for flame ai\videos for train\code_check"
process_all_videos(video_folder)


### Creating Vectors For Model

In [6]:

# Specify the directory paths
input_csv_folder = r"C:\Users\thaim\OneDrive\Desktop\test for flame ai\videos for train\interrupt_for_ai_machine"
output_vectors_dir = r"C:\Users\thaim\OneDrive\Desktop\test for flame ai\videos for train after code\Interrupt_Objects"

# Process all CSV files in the specified directory
process_all_csvs(input_csv_folder, output_vectors_dir)


### Creating Prepared Data File With Labels

In [22]:
folders = [
    r"C:\Users\thaim\OneDrive\Desktop\test for flame ai\videos for train after code\fire_1_features",
    r"C:\Users\thaim\OneDrive\Desktop\test for flame ai\videos for train after code\fire_2_features",
    r"C:\Users\thaim\OneDrive\Desktop\test for flame ai\videos for train after code\fire_3_features",
    r"C:\Users\thaim\OneDrive\Desktop\test for flame ai\videos for train after code\fire_4_features",
    r"C:\Users\thaim\OneDrive\Desktop\test for flame ai\videos for train after code\fire_5_features",
    r"C:\Users\thaim\OneDrive\Desktop\test for flame ai\videos for train after code\fire_6_features",
]

# Output directory for saving CSV files
output_dir = r"C:\Users\thaim\OneDrive\Desktop\test for flame ai\videos for train after code\Interrupt_Objects"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Loop through each folder and prepare data
for folder_path in folders:
    # Prepare data
    prepared_df = prepare_data(folder_path)
    
    # Extract folder name to use in the output file name
    folder_name = os.path.basename(folder_path)
    
    # Define the output file path
    output_file = os.path.join(output_dir, f"{folder_name}_prepared_data.csv")
    
    # Save prepared data to CSV
    prepared_df.to_csv(output_file, index=False)
    
    # Print a message indicating completion
    print(f"Saved prepared data to {output_file}")


Loading files for parameter: area_ratio


Processing area_ratio files: 100%|██████████| 47/47 [00:00<00:00, 497.97it/s]


Loading files for parameter: contour_area


Processing contour_area files: 100%|██████████| 47/47 [00:00<00:00, 558.98it/s]


Loading files for parameter: rect_area


...

Loading files for parameter: centerX


Processing centerX files: 100%|██████████| 223/223 [00:00<00:00, 447.13it/s]


Loading files for parameter: centerY


Processing centerY files: 100%|██████████| 223/223 [00:00<00:00, 536.90it/s]

Saved prepared data to C:\Users\thaim\OneDrive\Desktop\test for flame ai\videos for train after code\Interrupt_Objects\fire_6_features_prepared_data.csv


## Test codes

